In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# 재현성을 위한 랜덤 시드 설정
manual_seed = 999
torch.manual_seed(manual_seed)

# 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
class Generator(nn.Module):
    def __init__(self, nz, ngf, nc):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(nz, 256),  # 입력 잡음 벡터를 받음
            nn.ReLU(inplace=True),  # 활성화 함수: ReLU
            nn.Linear(256, 512),  # 은닉층
            nn.ReLU(inplace=True),  # 활성화 함수: ReLU
            nn.Linear(512, 784),  # MNIST 이미지 크기 28x28에 대한 출력
            nn.Tanh()  # 출력을 -1과 1 사이 값으로 조정
        )

    def forward(self, input):
        return self.main(input).view(-1, 1, 28, 28)

# 구분자 정의
class Discriminator(nn.Module):
    def __init__(self, nc, ndf):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(784, 512),  # 이미지를 1차원 벡터로 펼침
            nn.ReLU(inplace=True),  # 활성화 함수: ReLU
            nn.Linear(512, 256),  # 은닉층
            nn.ReLU(inplace=True),  # 활성화 함수: ReLU
            nn.Linear(256, 1)  # 하나의 스칼라 값으로 출력 (실제 또는 가짜)
        )

    def forward(self, input):
        input = input.view(-1, 784)  # 이미지를 1차원 벡터로 펼침
        return self.main(input)


In [4]:
# 파라미터 설정
batch_size = 64  # 배치 크기
nz = 100  # 잠재 벡터 크기
ngf = 64  # 생성자의 특징 맵 크기
ndf = 64  # 구분자의 특징 맵 크기
nc = 1  # 이미지 채널 수 (흑백 이미지이므로 1)
lr = 0.0002  # 학습률
epochs = 20  # 총 훈련 반복 횟수


In [5]:
# 생성자와 구분자 생성
netG = Generator(nz, ngf, nc).to(device)  # 생성자 객체 생성 및 GPU 또는 CPU에 로드
netD = Discriminator(nc, ndf).to(device)  # 구분자 객체 생성 및 GPU 또는 CPU에 로드

In [6]:
# 가중치 초기화 함수 정의
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

# 생성자와 구분자의 가중치 초기화
netG.apply(weights_init)
netD.apply(weights_init)

Discriminator(
  (main): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=256, out_features=1, bias=True)
  )
)

In [7]:
# 옵티마이저 생성
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(0.5, 0.999))  # 생성자용 Adam 옵티마이저
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(0.5, 0.999))  # 구분자용 Adam 옵티마이저

# 손실 함수 정의
criterion = nn.MSELoss()  # 손실 함수: 평균 제곱 오차

In [8]:
# 데이터 로딩
transform = transforms.Compose([
    transforms.ToTensor(),  # 이미지를 텐서로 변환
    transforms.Normalize((0.5,), (0.5,))  # 이미지를 -1에서 1로 정규화
])

dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)  # MNIST 데이터셋 로드
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)  # 데이터 로더 생성

In [9]:
# 생성된 이미지 저장 및 시각화를 위한 변수 초기화
img_list = []
fixed_noise = torch.randn(64, nz, device=device)

# 훈련 루프
for epoch in range(epochs):
    for i, data in enumerate(dataloader, 0):
        # 구분자 업데이트
        netD.zero_grad()  # 구분자의 그래디언트 초기화
        real_cpu = data[0].to(device)  # 실제 데이터 배치를 GPU 또는 CPU에 로드
        batch_size = real_cpu.size(0)  # 배치 크기 가져오기
        label = torch.full((batch_size,), 1, dtype=torch.float, device=device)  # 실제 이미지용 레이블

        output = netD(real_cpu).view(-1)  # 실제 이미지를 구분자에 전달하여 출력 계산
        errD_real = criterion(output, label)  # 실제 이미지에 대한 손실 계산
        errD_real.backward()  # 실제 이미지에 대한 그래디언트 계산
        D_x = output.mean().item()  # 실제 이미지에 대한 구분자의 평균 출력 계산

        noise = torch.randn(batch_size, nz, device=device)  # 잠재 벡터 생성